In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import unicodedata
import math
import os
import matplotlib.patches as mpatches
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import text
from tqdm import tqdm
from collections import Counter
import unicodedata
import re
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm
from collections import Counter
import huggingface_hub
import os
import warnings
import time

In [2]:
# TODO Do not duplicate code for this class
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        # TODO Do we have to do max norm here?
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, max_norm=1.0)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        x = self.embeddings(inputs)
        x = x.mean(dim=1)
        x = self.linear(x)
        return x

In [3]:
checkpoint = torch.load("cbow_final_with_vocab.pt", map_location=torch.device('cpu'))

word2idx = checkpoint['word2idx']
idx2word = checkpoint['idx2word']

vocab_size = len(word2idx)
embedding_dim = 100
model = CBOW(vocab_size, embedding_dim)

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [2]:
user = "sy91dhb"
password = "g5t49ao"
host = "178.156.142.230"
port = "5432"
db = "hd64m1ki"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")

In [3]:
query = "SELECT title, url, by, time, score FROM hacker_news.items WHERE type='story' AND dead IS NULL"
df = pd.read_sql(query, engine)

In [4]:
df.to_parquet("hackernews_stories.parquet")

In [5]:
df = pd.read_parquet("hackernews_stories.parquet")

In [18]:
class TitleWordIndicesDataset(Dataset):
    def __init__(self, dataframe, word2idx):
        self.samples = []
        for _, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Processing titles"):
            title = row['title']
            score_val = row['score']
            if not isinstance(title, str) or pd.isna(score_val):
                continue
            score = torch.tensor(score_val, dtype=torch.float32)
            words = title.lower().split()
            indices = [word2idx[w] for w in words if w in word2idx]
            if indices:
                self.samples.append((indices, score))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [19]:
# Sort the dataframe by time
df_sorted = df.sort_values('time')

# Calculate the split index
split_idx = int(len(df_sorted) * 0.8)

# Split into train and test
df_train = df_sorted.iloc[:split_idx].reset_index(drop=True)
df_test = df_sorted.iloc[split_idx:].reset_index(drop=True)

# Create datasets
title_indices_train = TitleWordIndicesDataset(df_train, word2idx)
title_indices_test = TitleWordIndicesDataset(df_test, word2idx)

Processing titles:   0%|          | 0/3413128 [00:00<?, ?it/s]

Processing titles:   0%|          | 0/853282 [00:00<?, ?it/s]

In [20]:
class TitleRegressionNN(nn.Module):
    def __init__(self, embedding_layer, embedding_dim, hidden_dim=128):
        super().__init__()
        # Use the pretrained embedding weights from the CBOW model
        self.embeddings = embedding_layer
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # Output is a single numeric value

    def forward(self, input_indices):
        # input_indices: (batch_size, seq_len)
        embeds = self.embeddings(input_indices)  # (batch_size, seq_len, embedding_dim)
        avg_embeds = embeds.mean(dim=1)         # (batch_size, embedding_dim)
        x = self.fc1(avg_embeds)
        x = self.relu(x)
        x = self.fc2(x)
        return x.squeeze(-1)  # (batch_size,)

model_reg = TitleRegressionNN(model.embeddings, embedding_dim)

In [21]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Device:", device)

Device: mps


In [23]:
# Training settings
batch_size = 128
num_epochs = 3
learning_rate = 1e-3

# Move model to device
model_reg = model_reg.to(device)

# Prepare DataLoader with padding for variable-length sequences
def collate_fn(batch):
    # batch is a list of (indices, score) tuples
    indices_list, scores_list = zip(*batch)
    lengths = [len(x) for x in indices_list]
    max_len = max(lengths)
    padded = [x + [0]*(max_len - len(x)) for x in indices_list]
    padded_indices = torch.tensor(padded, dtype=torch.long)
    scores = torch.tensor(scores_list, dtype=torch.float32)
    return padded_indices, scores

train_loader = DataLoader(title_indices_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Prepare target scores
train_scores = torch.tensor(df_train['score'].values, dtype=torch.float32)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_reg.parameters(), lr=learning_rate)

model_reg.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (batch_indices, batch_scores) in enumerate(train_loader):
        batch_indices, batch_scores = batch_indices.to(device), batch_scores.to(device)

        optimizer.zero_grad()
        outputs = model_reg(batch_indices)
        loss = criterion(outputs, batch_scores)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            avg_loss = running_loss / 100
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {avg_loss:.4f}")
            running_loss = 0.0

/opt/anaconda3/envs/ml/lib/python3.11/site-packages/torch/nn/functional.py:2434: UserWarning: The operator 'aten::embedding_renorm_' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:14.)
  torch.embedding_renorm_(weight.detach(), input, max_norm, norm_type)


Epoch [1/3], Step [100/25309], Loss: 3110.7890
Epoch [1/3], Step [200/25309], Loss: 3210.9402
Epoch [1/3], Step [300/25309], Loss: 3470.4803
Epoch [1/3], Step [400/25309], Loss: 3894.3878
Epoch [1/3], Step [500/25309], Loss: 2954.6800
Epoch [1/3], Step [600/25309], Loss: 4429.9621
Epoch [1/3], Step [700/25309], Loss: 4103.9661
Epoch [1/3], Step [800/25309], Loss: 3162.6637
Epoch [1/3], Step [900/25309], Loss: 4366.0110
Epoch [1/3], Step [1000/25309], Loss: 3855.9386
Epoch [1/3], Step [1100/25309], Loss: 3193.0165
Epoch [1/3], Step [1200/25309], Loss: 3010.4033
Epoch [1/3], Step [1300/25309], Loss: 5005.3872
Epoch [1/3], Step [1400/25309], Loss: 3720.7254
Epoch [1/3], Step [1500/25309], Loss: 3375.5183
Epoch [1/3], Step [1600/25309], Loss: 5407.9011
Epoch [1/3], Step [1700/25309], Loss: 2861.4455
Epoch [1/3], Step [1800/25309], Loss: 3071.5765
Epoch [1/3], Step [1900/25309], Loss: 3349.1587
Epoch [1/3], Step [2000/25309], Loss: 3192.9925
Epoch [1/3], Step [2100/25309], Loss: 4006.9798
E

KeyboardInterrupt: 

In [6]:
class HackerNewsDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Example: return title, score as tensor, and author
        # You can customize this as needed
        title = row['title']
        score = torch.tensor(row['score'], dtype=torch.float32)
        author = row['by']
        return {"title": title, "score": score, "author": author}

# Example usage:
hn_dataset = HackerNewsDataset(df)
hn_loader = DataLoader(hn_dataset, batch_size=32, shuffle=True)

In [7]:
# Sort the dataframe by time
df_sorted = df.sort_values('time')

# Calculate the split index
split_idx = int(len(df_sorted) * 0.8)

# Split into train and test
df_train = df_sorted.iloc[:split_idx].reset_index(drop=True)
df_test = df_sorted.iloc[split_idx:].reset_index(drop=True)

# Create datasets
hn_train = HackerNewsDataset(df_train)
hn_test = HackerNewsDataset(df_test)